In [ ]:
!pip install flask-ngrok
!pip install flask
!pip install pyngrok
!pip install line-bot-sdk
!ngrok authtoken 'Your ngrok Token' #你自己的ngrok Token
Line_Channel_Access_Token='Your Line_Channel_Access_Token'
Line_Channel_Secret='Your Line_Channel_Secret'

In [ ]:

#自動更新WebhookURL
def auto_update_webhook_url():
  time.sleep(5) #等候5秒讓ngrok完成註冊新網址
  try:
    import json
    import requests    
    #取得ngrok最新產生的url
    self_url = "http://localhost:4040/api/tunnels"
    res = requests.get(self_url)
    res_unicode = res.content.decode("utf-8")
    res_json = json.loads(res_unicode)  
    ngrok_url = res_json["tunnels"][0]["public_url"]

    #開始更新
    line_put_endpoint_url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    data = {"endpoint": ngrok_url + '/callback'}
    headers = {
      "Authorization": "Bearer " + Line_Channel_Access_Token ,
      "Content-Type": "application/json"
    }
    print(data)
    res = requests.put(line_put_endpoint_url, headers=headers, json=data)
    # 檢查回應狀態碼
    if res.status_code == 200:
      print("WebhookURL更新成功！")
    else:
      print("WebhookURL更新失敗")
    #強制關閉threading
    x._stop()
  except:
    pass


In [ ]:
from flask import *
from flask_ngrok import run_with_ngrok
from linebot import (LineBotApi, WebhookHandler)
from linebot.exceptions import ( InvalidSignatureError)
from linebot.models import *
import time
import threading

#Messaging API settings/Channel access token
line_bot_api = LineBotApi(Line_Channel_Access_Token)

#Basic settings/Channel secret 
handler = WebhookHandler(Line_Channel_Secret)

app = Flask(__name__)
run_with_ngrok(app)  



app = Flask(__name__)
run_with_ngrok(app)  

@app.route("/")
def home():
  return "<h1>我是大帥哥</h1>" 

@app.route("/callback", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

#實際處理訊息的部份
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):    
    replyMsg=[ TextSendMessage(text=event.message.text)]       #回覆文字=收到的文字
    line_bot_api.reply_message(
        event.reply_token,
        replyMsg
    )


if __name__ == "__main__":
  #啟動執行緒以更新webhook_url網址
  x = threading.Thread(target=auto_update_webhook_url)
  x.start()
  app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://xxxxxxxxxx.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
WebhookURL更新成功！
